In [1]:
import os 

from keras.models import Sequential 
from keras.layers import Flatten, Dense, Dropout, Cropping2D, Lambda, InputLayer
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras import losses, optimizers, metrics, backend, regularizers 
from keras.constraints import maxnorm
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [2]:
from typing import Iterator, List, Union, Tuple
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [3]:
dataset = ('C:/Users/Ketan/Thesis/self-driving-car/challenges/resized_dataset_udacity_320x240')

In [4]:
# Read the data
df = pd.io.parsers.read_csv(os.path.join(dataset, 'out.csv'))
# Split data into training and validation sets
df_train, df_val = train_test_split(df, test_size=.2)


In [5]:
datagen=ImageDataGenerator(rescale=1./255,zoom_range=0.2,horizontal_flip=True)

In [6]:
pd.options.mode.chained_assignment = None #default = warn

In [7]:
df_train['angle_range'] = pd.qcut(df_train['angle'], q=10)
df_val['angle_range'] = pd.qcut(df_val['angle'], q=10)

In [8]:
df_train['labels']= df_train['angle_range'].cat.codes
df_val['labels']= df_val['angle_range'].cat.codes

In [9]:
df_train['labels'] = df_train['labels'].apply(str)
df_val['labels'] = df_val['labels'].apply(str)


In [10]:
train_generator=datagen.flow_from_dataframe(dataframe=df_train, directory=dataset, x_col="File", y_col="labels", 
                                            class_mode="categorical", target_size=(240, 320), batch_size=32)

Found 4491 validated image filenames belonging to 10 classes.


In [11]:
val_generator=datagen.flow_from_dataframe(dataframe=df_val, directory=dataset, 
                                            x_col="File", y_col="labels", class_mode="categorical", target_size=(240, 320), 
                                            batch_size=32)

Found 1123 validated image filenames belonging to 10 classes.


In [12]:
def build_model(drop_prob = 0.2, learn_rate=0.001, constraint=maxnorm(3), reg = None, verbose = False): 
    backend.clear_session()
    model = Sequential()

    ###--- Convolutional Neural Network --- ### 
    input_shape = (240, 320, 3)
    model.add(InputLayer(input_shape=input_shape))

#     model.add(Cropping2D(cropping=((60, 0), (0, 0)), input_shape=input_shape))
    # model.add(Lambda(resize, input_shape=input_shape))
    model.add(BatchNormalization(name='r0'))

    ### conv. layers 
    model.add(Convolution2D(filters=24,
                            kernel_size=(5, 5),
                            strides = (2, 2),
                            activation='relu',
                            kernel_regularizer=reg, 
                            input_shape=input_shape, 
                            ))
    '''READ THIS try adding max pool - before min loss was 7.04'''
#     model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization(name='r1'))

    model.add(Convolution2D(filters=36,
                            kernel_size=(5, 5),
                            strides = (2, 2),
                            activation='relu',
                            kernel_regularizer=reg, 
                            ))
    model.add(BatchNormalization(name='r2'))

    model.add(Convolution2D(filters=48,
                            kernel_size=(5, 5),
                            strides = (2, 2),
                            activation='relu',
                            kernel_regularizer=reg, 
                            ))
    model.add(BatchNormalization(name='r3'))

    model.add(Convolution2D(filters=64,
                            kernel_size=(3, 3),
                            strides = (1, 1),
                            activation='relu',
                            kernel_regularizer=reg, 
                            ))
    model.add(BatchNormalization(name='r4'))

    model.add(Convolution2D(filters=64,
                            kernel_size=(1, 1),
                            strides = (2, 2),
                            activation='relu',
                            kernel_regularizer=reg, 
                            ))
    model.add(BatchNormalization(name='r5'))

    ### fully connect layers 
    model.add(Flatten())
    model.add(Dense(1164, activation='relu', kernel_constraint=constraint))
    model.add(Dropout(drop_prob))

    model.add(Dense(100, activation='relu', kernel_constraint=constraint))
    model.add(Dropout(drop_prob))

    model.add(Dense(50, activation='relu', kernel_constraint=constraint))
    model.add(Dropout(drop_prob))

    model.add(Dense(20, activation='relu', kernel_constraint=constraint))
    model.add(Dropout(drop_prob))

    model.add(Dense(10, activation='softmax'))

    if verbose: 
        # Summarize the model
        model.summary()
    
    #compile 
    model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=learn_rate),
                  metrics=['AUC', 'categorical_accuracy'])

    return model

In [16]:
#tensorboard object 
tensorboard = TensorBoard(log_dir='./logs7', histogram_freq=0,
                          write_graph=True, write_images=False)

#save and overwrite models when improved 
filepath="checkpoints/baseline_best.h5"

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', save_best_only=True)

early_stop = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3, verbose=0, mode='auto')

callbacks_list = [checkpoint, tensorboard, early_stop]

In [13]:
BATCH_SIZE = 32
EPOCHS = 10
LEARN_RATE = 0.001
DROP_PROB = 0.4

control_model = build_model(drop_prob = DROP_PROB, learn_rate = LEARN_RATE, verbose=True)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
r0 (BatchNormalization)      (None, 240, 320, 3)       12        
_________________________________________________________________
conv2d (Conv2D)              (None, 118, 158, 24)      1824      
_________________________________________________________________
r1 (BatchNormalization)      (None, 118, 158, 24)      96        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 57, 77, 36)        21636     
_________________________________________________________________
r2 (BatchNormalization)      (None, 57, 77, 36)        144       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 27, 37, 48)        43248     
_________________________________________________________________
r3 (BatchNormalization)      (None, 27, 37, 48)        1

C:\Users\Ketan\anaconda3\envs\tf-gpu-cuda\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [14]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=val_generator.n//val_generator.batch_size

In [17]:
control_hist = control_model.fit_generator(generator = train_generator,
                           steps_per_epoch = STEP_SIZE_TRAIN,
                           validation_data = val_generator,
                           validation_steps =STEP_SIZE_VALID, 
                           epochs = EPOCHS,
                           callbacks = callbacks_list,
                           verbose = 1)

C:\Users\Ketan\anaconda3\envs\tf-gpu-cuda\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
140/140 [==============================] - 301s 2s/step - loss: 2.5805 - auc: 0.5065 - categorical_accuracy: 0.1083 - val_loss: 2.3035 - val_auc: 0.5028 - val_categorical_accuracy: 0.1125
Epoch 2/10
140/140 [==============================] - 292s 2s/step - loss: 2.3129 - auc: 0.5050 - categorical_accuracy: 0.1041 - val_loss: 2.3024 - val_auc: 0.4985 - val_categorical_accuracy: 0.1098
Epoch 3/10
140/140 [==============================] - 321s 2s/step - loss: 2.3085 - auc: 0.5036 - categorical_accuracy: 0.0985 - val_loss: 2.2976 - val_auc: 0.5187 - val_categorical_accuracy: 0.1107
Epoch 4/10
140/140 [==============================] - 320s 2s/step - loss: 2.3149 - auc: 0.5115 - categorical_accuracy: 0.1054 - val_loss: 2.3020 - val_auc: 0.5033 - val_categorical_accuracy: 0.1027


In [40]:
control_model.save('resized_udacity_regression_v2.h5')


In [21]:
from keras.applications import inception_v3 as inc_net


In [22]:
from PIL import Image
import numpy as np

In [23]:
def open_images(inference_folder: str) -> np.ndarray:
    """Loads images from a folder and prepare them for inferencing.

    Parameters
    ----------
    inference_folder : str
        Location of images for inferencing.

    Returns
    -------
    np.ndarray
        List of images as numpy arrays transformed to fit the efficient_net model input specs.
    """
    images = []
    for img in os.listdir(inference_folder):
        img_location = os.path.join(inference_folder, img)  # create full path to image

        with Image.open(img_location) as img:  # open image with pillow

            img = np.array(img)
            img = img[:, :, :3]
            img = np.expand_dims(img, axis=0)  # add 0 dimension to fit input shape of efficient_net
            img = inc_net.preprocess_input(img)

        images.append(img)
    images_array = np.vstack(images)  # combine images efficiently to a numpy array
    return images_array

In [27]:
images = open_images("C:/Users/Ketan/Thesis/self-driving-car/challenges/test_dataset2")


In [41]:
predictions = control_model.predict_generator(images)

images_names = os.listdir("C:/Users/Ketan/Thesis/self-driving-car/challenges/test_dataset")
for image_name, prediction in zip(images_names, predictions):
    print(image_name, prediction)

1479425719681268780.jpg [-0.29937765]
1479425719731264417.jpg [-0.2768441]
1479425719781262693.jpg [-0.05526823]
1479425719831276356.jpg [-0.01969705]
1479425719881419974.jpg [0.13547224]
1479425719931414543.jpg [0.04343991]
1479425719981321507.jpg [0.61118335]
1479425720031300599.jpg [0.54303086]
1479425720081346893.jpg [0.44364884]
1479425720131269683.jpg [0.42185518]
1479425720181373910.jpg [0.6092737]
1479425720231334733.jpg [0.47977987]
1479425720281331591.jpg [0.8771286]
1479425720331302374.jpg [0.57536685]
1479425720381363046.jpg [0.8274342]
1479425720431335470.jpg [0.56299365]
1479425720481412110.jpg [1.9465857]
1479425720531376518.jpg [1.9624428]
1479425720581393603.jpg [2.1463106]
1479425720631398027.jpg [1.8737742]
1479425720681517212.jpg [1.9571255]
